# Example Use of AdvanceOrder

## Setup

In [1]:
import os
os.chdir(os.path.pardir)

# Testing classes
from src.market.spot import Spot
from src.market_actor import MarketActorStub
from src.market_listener import MarketListenerStub

In [2]:
transact_fee_rate = 0.2 / 100
stop_loss_rate = 2 / 100
trailing_rate = 1 / 100
entry_price = 1000

market = Spot("ETH", "USD")
market_listener = MarketListenerStub(market)
market_actor = MarketActorStub(transact_fee_rate)

## Trailing Stop Loss

In [3]:
from src.advance_order.trailing_stop_loss import TrailingStopLossLogic

trailing_stop_loss_logic = TrailingStopLossLogic(market_actor, market_listener, trailing_rate,
        use_orderbook=False)

### Long Position

In [4]:
long_position = market_actor.open_position(market, price=entry_price, size=10)
trailing_stop_loss = trailing_stop_loss_logic.open_advance_order(long_position)

f"trigger_price = {trailing_stop_loss.stop_loss_price}"
long_position.balances

Bought 10.00 units of ETH/USD    at 1000.00


{'ETH': 9.98, 'USD': -10000.0}

In [5]:
# 1. Not triggered: change in stop_loss_price
market_listener.set_current_price(1100)
trailing_stop_loss.update()

print(f"1. trigger_price = {trailing_stop_loss.stop_loss_price}")

# 2. Not triggered: no change in stop_loss_price
market_listener.set_current_price(1090)
trailing_stop_loss.update()

print(f"2. trigger_price = {trailing_stop_loss.stop_loss_price}")

# 3. Triggered
market_listener.set_current_price(1080)
trailing_stop_loss.update()

print(f"3. Order filled = {trailing_stop_loss.filled}")
# View ending position
trailing_stop_loss.position.balances

1. trigger_price = 1089.0
2. trigger_price = 1089.0
Sold   9.98 units of ETH/USD    at 1080.00
3. Order filled = True


{'ETH': 0.0, 'USD': 756.8431999999993}

### Short Position

In [6]:
short_position = market_actor.open_position(market, price=entry_price, size=-10)
trailing_stop_loss = trailing_stop_loss_logic.open_advance_order(short_position)

f"trigger_price = {trailing_stop_loss.stop_loss_price}"

Sold   10.00 units of ETH/USD    at 1000.00


'trigger_price = 1010.0'

In [7]:
# 1. Not triggered: change in stop_loss_price
market_listener.set_current_price(900)
trailing_stop_loss.update()

print(f"1. trigger_price = {trailing_stop_loss.stop_loss_price}")

# 2. Triggered
market_listener.set_current_price(950)
trailing_stop_loss.update()

print(f"2. Order filled = {trailing_stop_loss.filled}")

# View ending position
trailing_stop_loss.position.balances

1. trigger_price = 909.0
Bought 10.02 units of ETH/USD    at 950.00
2. Order filled = True


{'ETH': 0.0, 'USD': 460.9619238476953}

## Convertible Stop Loss

In [8]:
from src.advance_order.convertible_stop_loss import ConvertibleStopLossLogic

convertible_stop_loss_logic = ConvertibleStopLossLogic(market_actor, market_listener,
        stop_loss_offset=stop_loss_rate, trailing_rate=trailing_rate, offset_as_rate=True,
        use_orderbook=False)

### Long Position

In [9]:
long_position = market_actor.open_position(market, price=entry_price, size=10)
convertible_stop_loss = convertible_stop_loss_logic.open_advance_order(long_position)

print(f"trigger_price = {convertible_stop_loss.stop_loss_price}")
print(long_position.balances)
(1 - convertible_stop_loss.stop_loss_price / entry_price) * 100 # At stop loss rate

Bought 10.00 units of ETH/USD    at 1000.00
trigger_price = 980.0
{'ETH': 9.98, 'USD': -10000.0}


2.0000000000000018

In [10]:
# 1. Not triggered: change in stop_loss_price to trailing rate
market_listener.set_current_price(1050)
convertible_stop_loss.update()

print(f"1. trigger_price = {convertible_stop_loss.stop_loss_price}")
print(f"At trailing_rate: {(1 - convertible_stop_loss.stop_loss_price / 1050) * 100} %")

# 2. Triggered: 
market_listener.set_current_price(1020)
convertible_stop_loss.update()

print(f"2. Order filled = {convertible_stop_loss.filled}")
# View ending position
convertible_stop_loss.position.balances

1. trigger_price = 1039.5
At trailing_rate: 1.0000000000000009 %
Sold   9.98 units of ETH/USD    at 1020.00
2. Order filled = True


{'ETH': 0.0, 'USD': 159.24079999999958}